# PCG Chap 2. Dungeon Crawler

In [ ]:
import numpy as np
import logging
from random import randrange

In [ ]:
def grow_regions(array_map):
    if not len(array_map.shape) == 2:
        logging.error(f"Array map shall be 2 dimensional, actual shape is {array_map.shape}")

    nplus_map = np.zeros(array_map.shape)
    adj_coords = {(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0,-1), (1,-1)}
    for x in range(array_map.shape[0]):
        for y in range(array_map.shape[1]):
            if array_map[x][y] > 0:
                nplus_map[x][y] = array_map[x][y]
                for dx, dy in adj_coords:
                    if 0 <= x+dx < array_map.shape[0] and 0 <= y+dy < array_map.shape[1]:
                        if array_map[x+dx][y+dy] == 0:
                            nplus_map[x+dx][y+dy] = array_map[x][y]
    return nplus_map

def get_random_pois(dimension: int, n_pois: int, n_typeof_pois: int):
    if n_pois < 3:
        raise ValueError("number of pois shall be > 3, we Delaunay triangulate pois later")
    return [[randrange(dimension), randrange(dimension), randrange(n_typeof_pois)+2] for n in range(n_pois)]

In [ ]:
dimension = 16
array_map = np.zeros((dimension,dimension))
random_pois = get_random_pois(dimension=dimension, n_pois=5, n_typeof_pois=5)

for poi in random_pois:
    array_map[poi[0]][poi[1]] = poi[2]

# display(random_pois)
display(array_map)

In [ ]:
nplusone_array = grow_regions(array_map)
# nplusone_array = np.dot(array_map, nplusone_array)
nplusone_array

In [ ]:

def line_draw_bresenham(x0,y0,x1,y1):
    points_line = []
    dx = abs(x1 - x0)
    sx = 1 if x0 < x1 else -1
    dy = -abs(y1 - y0)
    sy = 1 if y0 < y1 else -1
    error = dx + dy
    while True:
        logging.debug(x0, y0)
        points_line.append((x0, y0))
        e2 = 2 * error
        if e2 >= dy:
            if x0 == x1:
                break
            error = error + dy
            x0 = x0 + sx
        if e2 <= dx:
            if y0 == y1:
                break
            error = error + dx
            y0 = y0 + sy
    return points_line

In [ ]:
# from itertools import combinations
# for duo in combinations(random_pois, 2):
#     print(duo)
from scipy.spatial import Delaunay

def get_all_points_in_lines(points):
    arr = np.asarray(points)
    # if not len(arr.shape == 2):
    #     return ValueError(f"Not in a good shape, should be points2D, shape={arr.shape}")
    tri = Delaunay(arr[:,:2])

    all_points_of_lines = set()
    for i, neighbors in enumerate(tri.neighbors):
        all_points_of_lines.add((points[i][0],points[i][1]))
        for n in neighbors:
            if n == -1:
                continue
            points_in_line = line_draw_bresenham(points[n][0],points[n][1],points[i][0],points[i][1])
            logging.debug(points_in_line)
            all_points_of_lines = all_points_of_lines.union(set(points_in_line))
    return all_points_of_lines

# arr = np.asarray(random_pois)
# arr.shape
# tri = Delaunay(arr[:,:2])

In [ ]:
# all_points_of_lines = set()
# for i, neighbors in enumerate(tri.neighbors):
#     for n in neighbors:
#         if n == -1:
#             continue
#         points_line = line_draw_bresenham(random_pois[n][0],random_pois[n][1],random_pois[i][0],random_pois[i][1])
#         logging.debug(points_line)
#         all_points_of_lines = all_points_of_lines.union(set(points_line))

In [ ]:
paths_map = np.zeros((dimension,dimension))
all_points_of_lines=get_all_points_in_lines(random_pois)
for p in all_points_of_lines:
    paths_map[p[0]][p[1]] = 1
paths_map

In [ ]:
myarr = np.asarray(random_pois)
up_random_pois=np.dot(myarr[:,:2],2)
up_all_points_of_lines=get_all_points_in_lines(up_random_pois)
up_paths_map = np.zeros((dimension*2,dimension*2))
for p in up_all_points_of_lines:
    up_paths_map[p[0]][p[1]] = 1
up_paths_map

In [ ]:
upscaled = np.full(np.dot(nplusone_array.shape, 2), -1)
for x in range(array_map.shape[0]):
    for y in range(array_map.shape[1]):
        upscaled[x*2][y*2] = nplusone_array[x][y]
# display(upscaled)

upscaled_final=np.copy(upscaled)
for x in range(0, dimension*2, 2):
    for y in range(0, dimension*2-1):
        if upscaled_final[x][y] == -1:
            if up_paths_map[x][y] == 1:
                upscaled_final[x][y] = 10
            if upscaled_final[x][y-1] == upscaled_final[x][y+1]:
                # print(f'equal {upscaled_pathed[x][y-1]}')
                upscaled_final[x][y] = upscaled_final[x][y-1]

for x in range(0, dimension*2-1):
    for y in range(dimension*2):
        if upscaled_final[x][y] == -1:
            if up_paths_map[x][y] == 1:
                upscaled_final[x][y] = 10
            if upscaled_final[x-1][y] == upscaled_final[x+1][y]:
                # print(f'equal {upscaled_pathed[x][y-1]}')
                upscaled_final[x][y] = upscaled_final[x-1][y]  

upscaled_final

In [ ]:
for p in up_random_pois:
    upscaled_final[p[0]][p[1]] = upscaled_final[p[0]][p[1]]*10

In [ ]:
upscaled_final

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('_mpl-gallery-nogrid')
# make data
X, Y = np.meshgrid(np.linspace(0, dimension*2), np.linspace(0, dimension*2))
Z = upscaled_final

# plot
fig, ax = plt.subplots()

ax.imshow(Z)

plt.show()